In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/CS114/Project/data

/content/drive/MyDrive/CS114/Project/data


In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 29.6 MB/s eta 0:00:00


# Import

In [ ]:
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import urllib.request
from tqdm.notebook import tqdm
import time
import random
import csv
import traceback
from bs4 import BeautifulSoup
import os
import sys
from urllib.parse import urljoin


In [ ]:
def simulate(actions, driver):  # Simulate human wait and mouse movement
    time.sleep(random.uniform(1.5, 2.1))
    try:
        # Scroll to top to ensure elements are in view before interacting
        driver.execute_script("window.scrollTo(0, 0);")
        # actions.move_by_offset(random.randint(150, 250), random.randint(150, 250)).perform()
        # actions.move_to_element_with_offset(driver.find_element(By.TAG_NAME, 'body'), 0, 0).perform()
    except Exception as e:
        print("Error in simulate:", e)

chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Remove Navigator.Webdriver Flag in Selenium
chrome_options.add_argument("--incognito")  # So that they can't track with cookie, session
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model (useful on servers)
chrome_options.add_argument("--disable-dev-shm-usage")  # Reduce shared memory usage
chrome_options.add_argument("--remote-debugging-port=9222")  # Enable remote debugging

#user agent for : linux, macos, win 7 ,8, 8.1, avoiding 10 cos mine, try 11 too
ua_list = [ "Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36", #win <8.1 doesnt officialy support chrome 116
            "Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"]

In [ ]:
random_user_agent = random.choice(ua_list)
chrome_options.add_argument(f"user-agent={random_user_agent}")

ua_choice = random.choice(ua_list)
chrome_options.add_argument(f'--user-agent={ua_choice}')
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(10)  # Set implicit wait for elements to load
actions = ActionChains(driver)

# Honda
Nguồn https://automobiles.honda.com/

In [ ]:
gallery_links = [
  'https://automobiles.honda.com/hr-v#gallery-exterior',
  'https://automobiles.honda.com/cr-v#gallery-exterior',
  'https://automobiles.honda.com/pilot#gallery-exterior',
  'https://automobiles.honda.com/passport#gallery-exterior',
  'https://automobiles.honda.com/prologue#gallery-exterior',
  'https://automobiles.honda.com/cr-v-fcev#gallery-exterior',
  'https://automobiles.honda.com/civic-sedan#gallery-exterior',
  'https://automobiles.honda.com/accord-sedan#gallery-exterior' ,
  'https://automobiles.honda.com/civic-si-sedan#gallery-exterior',
  'https://automobiles.honda.com/civic-hatchback#gallery-exterior',
  'https://automobiles.honda.com/civic-type-r#gallery-exterior',
  'https://automobiles.honda.com/ridgeline#gallery-exterior',
  'https://automobiles.honda.com/odyssey#gallery-exterior'
]

In [ ]:
def crawl_images(driver, gallery_url):
    base_url = "https://automobiles.honda.com"
    driver.get(gallery_url)
    simulate(actions, driver)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    li_elements = driver.find_elements(By.CLASS_NAME, "hrt-responsive-background-behavior")
    image_urls = []
    for li in li_elements:
        img_large = li.get_attribute("data-full-img-large-2x")
        if img_large:
            img_large = img_large.split("?")[0]  # Cắt phần query
            if img_large.startswith("/"):
                image_urls.append(base_url + img_large)
    return image_urls

In [ ]:
output_folder = "./honda"
image_name_counter = 1
try:
    images_links = []
    for gallery_link in gallery_links:
        images = crawl_images(driver, gallery_link)
        for img in images:
            images_links.append(img)
finally:
    driver.quit()

In [ ]:
print("Số lượng link ảnh Honda: ",len(images_links))

Số lượng link ảnh Honda:  375


In [ ]:
print(images_links[:10])

['https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/01/Modal/MY25-hr-v-exterior-gallery-01-1400-2x.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/02/Modal/MY25-HR-V-gallery-01-1400-2x.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/03/Modal/MY25-HR-V-gallery-03-1400-2x.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/04/Modal/MY25-HR-V-gallery-04-1400-2x.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/05/Modal/MY25-HR-V-gallery-05-1400-2x.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/07/Modal/MY25-HR-V-gallery-07-1400-2x.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/08/Modal/MY25-HR-V-gallery-08-1400-2x-A.jpg', 'https://automobiles.honda.com/-/media/Honda-Automobiles/Vehicles/2025/HR-V/Gallery/09/Modal

In [ ]:
def download_image_selenium(driver, image_url, output_folder, image_name_counter):
    try:
        driver.get(image_url)
        os.makedirs(output_folder, exist_ok=True)
        file_path = os.path.join(output_folder, f"22520394.22520395.Honda.{image_name_counter}.jpg")

        img = driver.find_element(By.TAG_NAME, "img")
        with open(file_path, 'wb') as f:
            f.write(img.screenshot_as_png)

        print(f"Downloaded via Selenium: {file_path}")
    except Exception as e:
        print(f"Error downloading image via Selenium: {e}")

output_folder = "./Honda"
image_name_counter = 1

try:
    for img in images_links:
        try:
            download_image_selenium(driver, img, output_folder, image_name_counter)
            image_name_counter += 1
        except Exception as e:
            print(f"Error with URL {img}: {e}")
        time.sleep(2)  # Thêm delay
finally:
    driver.quit()


Downloaded via Selenium: ./honda/22520394.22520395.Honda.1.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.2.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.3.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.4.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.5.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.6.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.7.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.8.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.9.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.10.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.11.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.12.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.13.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.14.jpg
Downloaded via Selenium: ./honda/22520394.22520395.Honda.15.jpg
Downloaded via Selenium: ./honda/22520394.2252039

# Toyota

Nguồn https://xe.chotot.com/mua-ban-oto


In [ ]:
from selenium.webdriver.common.by import By
import time

def extract_urls(driver, car_url):
    base_url = "https://xe.chotot.com"
    driver.get(car_url)
    simulate(actions, driver)
    time.sleep(10)  # Đợi trang tải

    # Cuộn trang để tải nội dung động
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(10)

    urls = []
    # Tìm tất cả các thẻ <a> có class="cebeqpz"
    a_elements = driver.find_elements(By.CSS_SELECTOR, "a.cebeqpz")
    print(f"Found {len(a_elements)} <a> elements with class='cebeqpz'.")

    for a_element in a_elements:
        href = a_element.get_attribute("href")  # Lấy thuộc tính href
        if href:
            href = href.split("#")[0]
            href = href.split("?")[0]
            if href.startswith("/"):  # Nếu là đường dẫn tương đối
                href = base_url + href
            urls.append(href)
            print(f"Extracted URL: {href}")

    return urls

# Gọi hàm
car_url = "https://xe.chotot.com/mua-ban-oto-toyota-sdcb2"
urls = extract_urls(driver, car_url)
print(f"Extracted {len(urls)} URLs")


Found 30 <a> elements with class='cebeqpz'.
Extracted URL: https://xe.chotot.com/mua-ban-oto-thanh-pho-thuan-an-binh-duong/121191993.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-thanh-pho-thuan-an-binh-duong/121191993.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-thanh-pho-thuan-an-binh-duong/121191993.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-tan-phu-tp-ho-chi-minh/121446279.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-tan-phu-tp-ho-chi-minh/121446279.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-tan-phu-tp-ho-chi-minh/121446279.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-12-tp-ho-chi-minh/121517717.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-12-tp-ho-chi-minh/121517717.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-12-tp-ho-chi-minh/121517717.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-hoang-mai-ha-noi/120994588.htm?ad_delivery=true&event_source=ad_list
Extracted URL: https://

In [ ]:
def crawl_images(driver, urls):
    driver.get(urls)
    simulate(actions, driver)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    div_elements = driver.find_elements(By.CLASS_NAME, "pzvfhqt")
    image_urls = []
    for div in div_elements:
      img_elements = div.find_elements(By.TAG_NAME, "img")
      for img in img_elements:
        img_url = img.get_attribute("src")
        if img_url and img_url.startswith("http"):
          image_urls.append(img_url)
          print(img_url)

    return image_urls
url_try = 'https://xe.chotot.com/mua-ban-oto-huyen-dong-anh-ha-noi/120889807.htm'
image_urls = crawl_images(driver, url_try)

https://xe.chotot.com/_next/image?url=https%3A%2F%2Fcdn.chotot.com%2FnVQMQsjATaElB2vdsunJpCvvuRXhwypIwZSheQo64Is%2Fpreset%3Aview%2Fplain%2F6df3ea1fe053d714f2c61903315d02aa-2904655062125107143.jpg&w=3840&q=100
https://xe.chotot.com/_next/image?url=https%3A%2F%2Fcdn.chotot.com%2Fs2jj4aEnNMtf0_mdEcvwXRThVkRDCWf9jDuO2BOsEug%2Fpreset%3Aview%2Fplain%2Fbcb34ec6bb4d19d2b03bdfa36e668a37-2904655062208986142.jpg&w=3840&q=100
https://xe.chotot.com/_next/image?url=https%3A%2F%2Fcdn.chotot.com%2F6hA5BbHmYGPNAkDUldPnzlIran8tHy5lWE6Hwm3qTUg%2Fpreset%3Aview%2Fplain%2F1a40ed48c13da6df281281b3474eb738-2904655061547250143.jpg&w=3840&q=100
https://xe.chotot.com/_next/image?url=https%3A%2F%2Fcdn.chotot.com%2FSwKA2VMDdsZ6MHAsGe_WDz8jxm0weM4UzVn1lydbzK0%2Fpreset%3Aview%2Fplain%2Fb4fe2de772439ab9e4f1bc0dfa205a1b-2904655062419730911.jpg&w=3840&q=100
https://xe.chotot.com/_next/image?url=https%3A%2F%2Fcdn.chotot.com%2FAWZG43rcq6-Cqyre7_H-uAZgn2QjQA7B6YXhRVJHZl4%2Fpreset%3Aview%2Fplain%2F272bf6b3d92ce346561739052

In [ ]:
import os
import requests

def download_image(image_url, output_folder, image_name_counter):
    """Tải ảnh trực tiếp bằng requests"""
    try:
        os.makedirs(output_folder, exist_ok=True)
        file_path = os.path.join(output_folder, f"22520394.22520395.Toyota.{image_name_counter}.jpg")
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)

        print(f"Downloaded: {file_path}")
    except Exception as e:
        print(f"Error downloading {image_url}: {e}")

# Thư mục lưu ảnh
output_folder = "./Toyota"
image_name_counter = 1

# Tải từng ảnh trong danh sách
for img_url in image_urls:
    download_image(img_url, output_folder, image_name_counter)
    image_name_counter += 1


Downloaded: ./Toyota/22520394.22520395.Toyota.1.jpg
Downloaded: ./Toyota/22520394.22520395.Toyota.2.jpg
Downloaded: ./Toyota/22520394.22520395.Toyota.3.jpg
Downloaded: ./Toyota/22520394.22520395.Toyota.4.jpg
Downloaded: ./Toyota/22520394.22520395.Toyota.5.jpg
Downloaded: ./Toyota/22520394.22520395.Toyota.6.jpg
Downloaded: ./Toyota/22520394.22520395.Toyota.7.jpg


# Các xe Toyota, Mitsubishi, Mazda, Kia, Hyundai, Honda, VinFast
Nguồn: https://xe.chotot.com

In [ ]:
url = 'https://xe.chotot.com/mua-ban-oto'
titles = ['Toyota', 'Mitsubishi', 'Mazda', 'Kia', 'Hyundai', 'Honda', 'VinFast']
car_urls = [
    'https://xe.chotot.com/mua-ban-oto-toyota-sdcb2',
    'https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18',
    'https://xe.chotot.com/mua-ban-oto-mazda-sdcb7',
    'https://xe.chotot.com/mua-ban-oto-kia-sdcb1',
    'https://xe.chotot.com/mua-ban-oto-hyundai-sdcb5',
    'https://xe.chotot.com/mua-ban-oto-honda-sdcb6',
    'https://xe.chotot.com/mua-ban-oto-vinfast-sdcb80'
]
car_links = dict(zip(titles, car_urls))
print(car_links)

{'Toyota': 'https://xe.chotot.com/mua-ban-oto-toyota-sdcb2', 'Mitsubishi': 'https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18', 'Mazda': 'https://xe.chotot.com/mua-ban-oto-mazda-sdcb7', 'Kia': 'https://xe.chotot.com/mua-ban-oto-kia-sdcb1', 'Hyundai': 'https://xe.chotot.com/mua-ban-oto-hyundai-sdcb5', 'Honda': 'https://xe.chotot.com/mua-ban-oto-honda-sdcb6', 'VinFast': 'https://xe.chotot.com/mua-ban-oto-vinfast-sdcb80'}


In [ ]:
def extract_urls(driver, car_url):
    base_url = "https://xe.chotot.com"
    driver.get(car_url)
    simulate(actions, driver)
    time.sleep(10)  # Đợi trang tải

    # Cuộn trang để tải nội dung động
    for _ in range(5):
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(5)

    urls = set()
    a_elements = driver.find_elements(By.CSS_SELECTOR, "a.cebeqpz")

    for a_element in a_elements:
      href = a_element.get_attribute("href")
      if href:
        href = href.split("#")[0]
        href = href.split("?")[0]
        href = urljoin(base_url, href)
        urls.add(href)
        print(f"Extracted URL: {href}")
    return list(urls)

def crawl_images(driver, post_url):
    driver.get(post_url)
    simulate(actions, driver)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    div_elements = driver.find_elements(By.CLASS_NAME, "pzvfhqt")
    image_urls = []
    for div in div_elements:
        img_elements = div.find_elements(By.TAG_NAME, "img")
        for img in img_elements:
            img_url = img.get_attribute("src")
            if img_url and img_url.startswith("http"):
                image_urls.append(img_url)
    return image_urls

def download_image(image_url, output_folder, image_name_counter, car_title):
    try:
        os.makedirs(output_folder, exist_ok=True)
        file_path = os.path.join(output_folder, f"{car_title}.{image_name_counter}.jpg")
        response = requests.get(image_url, stream=True)
        response.raise_for_status()

        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
    except Exception as e:
        print(f"Error downloading {image_url}: {e}")

try:
  for car_title, car_url in car_links.items():
    print(f"Processing car type: {car_title}")
    output_folder = f"./{car_title}"
    post_urls = extract_urls(driver, car_url)
    if not post_urls:
      print(f"No posts found for {car_title}. Skipping...")
      continue
    image_name_counter = 1
    total_images = 0
    for post_url in post_urls:
      try:
        image_urls = crawl_images(driver, post_url)
        if not image_urls:
          print(f"No images found in post: {post_url}")
          continue
        for img_url in image_urls:
          download_image(img_url, output_folder, image_name_counter, car_title)
          image_name_counter += 1
          total_images += 1
      except Exception as e:
        print(f"Error processing post {post_url}: {e}")
    print(f"Total images downloaded for {car_title}: {total_images}")
finally:
    driver.quit()

Processing car type: Toyota
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/121395092.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/121395092.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/121395092.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-cai-rang-can-tho/121191132.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-cai-rang-can-tho/121191132.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-cai-rang-can-tho/121191132.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-go-vap-tp-ho-chi-minh/116480847.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-tan-binh-tp-ho-chi-minh/121187854.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-ninh-kieu-can-tho/120956340.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-hoang-mai-ha-noi/121002040.htm
Extracted URL: https://xe.chotot.com/mua-ban-oto-quan-hoang-mai-ha-noi/121191936.htm
Extracted

####

# Bổ xung Toyota, Mitsubishi, Mazda, Kia, Hyundai, Honda, VinFast, Suzuki

In [ ]:
%cd /content/drive/MyDrive/CS114/DoAn/data

/content/drive/MyDrive/CS114/DoAn/data


In [ ]:
url = 'https://xe.chotot.com/mua-ban-oto'
titles = ['Toyota', 'Mitsubishi', 'Mazda', 'Kia', 'Hyundai', 'Honda', 'VinFast', 'Suzuki']
car_urls = [
    'https://xe.chotot.com/mua-ban-oto-toyota-sdcb2',
    'https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18',
    'https://xe.chotot.com/mua-ban-oto-mazda-sdcb7',
    'https://xe.chotot.com/mua-ban-oto-kia-sdcb1',
    'https://xe.chotot.com/mua-ban-oto-hyundai-sdcb5',
    'https://xe.chotot.com/mua-ban-oto-honda-sdcb6',
    'https://xe.chotot.com/mua-ban-oto-vinfast-sdcb80',
    'https://xe.chotot.com/mua-ban-oto-suzuki-sdcb22'
]
car_links = dict(zip(titles, car_urls))
print(car_links)

{'Toyota': 'https://xe.chotot.com/mua-ban-oto-toyota-sdcb2', 'Mitsubishi': 'https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18', 'Mazda': 'https://xe.chotot.com/mua-ban-oto-mazda-sdcb7', 'Kia': 'https://xe.chotot.com/mua-ban-oto-kia-sdcb1', 'Hyundai': 'https://xe.chotot.com/mua-ban-oto-hyundai-sdcb5', 'Honda': 'https://xe.chotot.com/mua-ban-oto-honda-sdcb6', 'VinFast': 'https://xe.chotot.com/mua-ban-oto-vinfast-sdcb80', 'Suzuki': 'https://xe.chotot.com/mua-ban-oto-suzuki-sdcb22'}


In [ ]:
from urllib.parse import urljoin

def extract_urls_from_pages(driver, base_url, max_pages):
  urls = set()
  for page in range(1, max_pages + 1):
    paginated_url = f"{base_url}?page={page}"
    print(f"Fetching page {page}: {paginated_url}")
    driver.get(paginated_url)
    simulate(actions, driver)
    time.sleep(5)  # Đợi trang tải
    for _ in range(5):
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      time.sleep(2)
    a_elements = driver.find_elements(By.CSS_SELECTOR, "a.cebeqpz")
    if not a_elements:  # Nếu không có thêm URL nào, dừng lặp
      print(f"No more URLs found on page {page}. Stopping...")
      break
    for a_element in a_elements:
      href = a_element.get_attribute("href")
      if href:
        href = href.split("#")[0]
        href = href.split("?")[0]
        href = urljoin(base_url, href)
        urls.add(href)
        # print(f"Extracted URL: {href}")
  return list(urls)


In [ ]:
try:
  for car_title, car_url in car_links.items():
    print(f"Processing car type: {car_title}")
    output_folder = f"./{car_title}"
    post_urls = extract_urls_from_pages(driver, car_url, max_pages=5)
    if not post_urls:
      print(f"No posts found for {car_title}. Skipping...")
      continue
    image_name_counter = 1
    total_images = 0
    for post_url in post_urls:
      try:
        image_urls = crawl_images(driver, post_url)
        if not image_urls:
          print(f"No images found in post: {post_url}")
          continue
        for img_url in image_urls:
          download_image(img_url, output_folder, image_name_counter, car_title)
          image_name_counter += 1
          total_images += 1
      except Exception as e:
        print(f"Error processing post {post_url}: {e}")
    print(f"Total images downloaded for {car_title}: {total_images}")
finally:
    driver.quit()

Processing car type: Toyota
Fetching page 1: https://xe.chotot.com/mua-ban-oto-toyota-sdcb2?page=1
Fetching page 2: https://xe.chotot.com/mua-ban-oto-toyota-sdcb2?page=2
Fetching page 3: https://xe.chotot.com/mua-ban-oto-toyota-sdcb2?page=3
Fetching page 4: https://xe.chotot.com/mua-ban-oto-toyota-sdcb2?page=4
Fetching page 5: https://xe.chotot.com/mua-ban-oto-toyota-sdcb2?page=5
Total images downloaded for Toyota: 634
Processing car type: Mitsubishi
Fetching page 1: https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18?page=1
Fetching page 2: https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18?page=2
Fetching page 3: https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18?page=3
Fetching page 4: https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18?page=4
Fetching page 5: https://xe.chotot.com/mua-ban-oto-mitsubishi-sdcb18?page=5
Total images downloaded for Mitsubishi: 644
Processing car type: Mazda
Fetching page 1: https://xe.chotot.com/mua-ban-oto-mazda-sdcb7?page=1
Fetching page 2: https:/

# Bổ xung

## Toyota

In [ ]:
image_urls = [
    'https://www.carpixel.net/w/0d3989256f7f651260268b350116d695/toyota-corolla-hybrid-wallpaper-hd-88894.jpg',
    'https://www.carpixel.net/w/afd73de21d3db061b84b7f831b051f90/toyota-corolla-hybrid-wallpaper-hd-88889.jpg',
    'https://www.carpixel.net/w/4f20adf33bc70f7270a3e566c9d948f8/toyota-corolla-hybrid-wallpaper-hd-88892.jpg',
    'https://www.carpixel.net/w/e5a94edf3a64069a6f8c3b7085e37582/toyota-corolla-hybrid-wallpaper-hd-88895.jpg',
    'https://www.carpixel.net/w/a29f2da4a819c514a22163bfae5c149c/toyota-corolla-hybrid-wallpaper-hd-88890.jpg',
    'https://www.carpixel.net/w/2d5a48297bb9d395be377a4d7f8b1f02/toyota-corolla-hybrid-sedan-wallpaper-hd-84817.jpg',
    'https://www.carpixel.net/w/ff3be7c6b9ad6efd49b1274630911891/toyota-corolla-hoonigan-wallpaper-hd-81257.jpg',
    'https://www.carpixel.net/w/f7711aadccef46ee9403280c8699bbc9/toyota-corolla-hoonigan-wallpaper-hd-81256.jpg',
    'https://www.carpixel.net/w/1d169c6aa3fe95937c394cf94e4b344e/toyota-corolla-hoonigan-wallpaper-hd-81255.jpg',
    'https://www.carpixel.net/w/07db36fc8d5daefe84820835be20db53/toyota-corolla-sedan-xse-wallpaper-hd-81628.jpg',
    'https://www.carpixel.net/w/2e40239312d5b6f15cc94942d3a838f7/toyota-corolla-sedan-xse-wallpaper-hd-81629.jpg',
    'https://www.carpixel.net/w/36cdb095be5d63c2f6d44c3b850556a4/toyota-corolla-sedan-xse-wallpaper-hd-81631.jpg',
    'https://www.carpixel.net/w/99503e4f41e7034d5cd3b65ad537f12b/toyota-corolla-hybrid-wallpaper-hd-88901.jpg',
    'https://www.carpixel.net/w/9fca84000e97d2cf86150480bc555561/toyota-corolla-hybrid-wallpaper-hd-88905.jpg',
    'https://www.carpixel.net/w/e0987b75b9799077ed49f14e7ba89993/toyota-camry-hybrid-sport-styling-wallpaper-hd-100086.jpg',
    'https://www.carpixel.net/w/6c50b3caff9023aff13b2838e61c76cb/toyota-camry-hybrid-sport-styling-wallpaper-hd-100087.jpg',
    'https://www.carpixel.net/w/9a87b4fb4bbd7308b4b543ea5bd317c0/toyota-camry-hybrid-sport-styling-wallpaper-hd-100088.jpg',
    'https://www.carpixel.net/w/89a59144f19a0fbaf91685eebb23f327/toyota-camry-hybrid-sport-styling-wallpaper-hd-100089.jpg'
]

## Hyundai

In [ ]:
# Hyundai
image_urls =[
    'https://www.carpixel.net/w/1d6da6afe5024a3608a14774ed5a3ebd/hyundai-accent-5-door-wallpaper-hd-3877.jpg',
    'https://www.carpixel.net/w/b48583798e62ff02eeeefa531238284c/hyundai-accent-5-door-wallpaper-hd-3878.jpg',
    'https://www.carpixel.net/w/ae0cc1796b764d9c8ed6ae17fb7552f2/hyundai-accent-5-door-wallpaper-hd-3881.jpg',
    'https://www.carpixel.net/w/3fce6ca2764e474050c419152b31e770/hyundai-tucson-xrt-wallpaper-hd-121850.jpg',
    'https://www.carpixel.net/w/a1946a731dd3f8205c85cdb9c5277708/hyundai-tucson-xrt-wallpaper-hd-121852.jpg',
    'https://www.carpixel.net/w/97d629cf20b2de03f8c971fe4834ee13/hyundai-tucson-xrt-wallpaper-hd-121851.jpg',
    'https://www.carpixel.net/w/0a06b14d2b672512a0b1b067787c41ff/hyundai-tucson-xrt-wallpaper-hd-121849.jpg',
    'https://www.carpixel.net/w/c129ea97a72dbee4db052b93c04af18a/hyundai-tucson-xrt-wallpaper-hd-121848.jpg',
    'https://www.carpixel.net/w/628f0748463fbaacdcfc751c254abc2b/hyundai-tucson-n-line-wallpaper-hd-121836.jpg',
    'https://www.carpixel.net/w/1033c73537e193f441c2e4c4b11159a5/hyundai-ioniq-5-n-wallpaper-hd-121513.jpg',
    'https://www.carpixel.net/w/0d33b3e819944de96a76243e27d281dc/hyundai-ioniq-5-n-wallpaper-hd-121512.jpg',
    'https://www.carpixel.net/w/e5f848f7203f472fb4ea89f5b9d23887/hyundai-kona-electric-wallpaper-hd-121123.jpg',
    'https://www.carpixel.net/w/6a6ab615b70506dcbfca6f9011b1fe83/hyundai-kona-electric-wallpaper-hd-121126.jpg',
    'https://www.carpixel.net/w/fd8b30fe633899640029c5001fd14ca4/hyundai-kona-electric-wallpaper-hd-121125.jpg',
    'https://www.carpixel.net/w/74ab471d3af6cdfd14cda18e4a96abb0/hyundai-kona-electric-wallpaper-hd-121122.jpg',
    'https://www.carpixel.net/w/2808f78af4c96b834647e6809a7ddfee/hyundai-kona-electric-n-line-wallpaper-hd-121127.jpg',
    'https://www.carpixel.net/w/2ef079fdcfedfbedb9439a9bc1196a97/hyundai-kona-electric-n-line-wallpaper-hd-121129.jpg',
    'https://www.carpixel.net/w/81fd8e4b9d1296852505e6614e4bde2b/hyundai-sonata-hybrid-wallpaper-hd-119879.jpg',
    'https://www.carpixel.net/w/50b760b6b482decf46af94b12c107674/hyundai-sonata-hybrid-wallpaper-hd-119880.jpg',
    'https://www.carpixel.net/w/9bf621f0d03f3f27e8b37e5629acedd7/hyundai-sonata-hybrid-wallpaper-hd-119876.jpg',
    'https://www.carpixel.net/w/e621da7dd547752e1c2d7d6a1984dc0b/hyundai-kona-wallpaper-hd-118179.jpg',
    'https://www.carpixel.net/w/ba83a1abee9fdf11a26e1b8e27bb6096/hyundai-kona-wallpaper-hd-118182.jpg',
    'https://www.carpixel.net/w/f2a97eb8a7516a44b520397ce439c5cb/hyundai-kona-wallpaper-hd-118181.jpg',
    'https://www.carpixel.net/w/f0fff6ddd7a200fb1de43649fe3c1f30/hyundai-kona-wallpaper-hd-118183.jpg',
    'https://www.carpixel.net/w/d14e4cd520399d870c6ba6470d1cb907/hyundai-kona-wallpaper-hd-118180.jpg',
    'https://www.carpixel.net/w/0f198514135e102e4f66a51ef0ac263c/hyundai-kona-electric-wallpaper-hd-118172.jpg',
    'https://www.carpixel.net/w/390b27546f2f62141ca335c1628365f2/hyundai-kona-electric-wallpaper-hd-118173.jpg',
    'https://www.carpixel.net/w/4ee4a59c07d6bb1c1fca973425dde421/hyundai-kona-electric-wallpaper-hd-118175.jpg',
    'https://www.carpixel.net/w/a27915c43234e39dac9445816394d4ad/hyundai-kona-electric-wallpaper-hd-118174.jpg',
    'https://www.carpixel.net/w/b42305ed5bc1b98eb2b0e6e0fb84a12a/hyundai-kona-electric-wallpaper-hd-118177.jpg',
    'https://www.carpixel.net/w/5f510fad0f08a1018ff48c14f1b9316d/hyundai-kona-n-line-wallpaper-hd-120020.jpg',
    'https://www.carpixel.net/w/ef2e80bb92712c1e34157ec493dcd7c5/hyundai-kona-n-line-wallpaper-hd-120018.jpg',
    'https://www.carpixel.net/w/a5a0258fd640ce67847d44ec20de8091/hyundai-kona-n-line-wallpaper-hd-120019.jpg',
    'https://www.carpixel.net/w/a60925cd131a697ffb2b19ec74ed0461/hyundai-santa-fe-hybrid-wallpaper-hd-119578.jpg',
    'https://www.carpixel.net/w/9b4fd3b9f81232b0be7ae765ddf312cd/hyundai-santa-fe-hybrid-wallpaper-hd-119577.jpg',
    'https://www.carpixel.net/w/c873e4237ba6eeff25b4cca51004706b/hyundai-ioniq-6-wallpaper-hd-118084.jpg',
    'https://www.carpixel.net/w/044412bf62b11cc248bec95fbcbee269/hyundai-ioniq-6-wallpaper-hd-118086.jpg',
    'https://www.carpixel.net/w/63794bdff319f2a5c0b88316ebe490bb/hyundai-ioniq-6-wallpaper-hd-118087.jpg',
    'https://www.carpixel.net/w/2c5ae6080b8c04c26ce7e86302d9d3ce/hyundai-ioniq-6-wallpaper-hd-118088.jpg'
]

In [ ]:
def download_image(image_url, output_folder, image_name_counter, car_title):
  try:
    os.makedirs(output_folder, exist_ok=True)
    file_path = os.path.join(output_folder, f"{car_title}.{image_name_counter}.jpg")
    response = requests.get(image_url, stream=True)
    response.raise_for_status()

    with open(file_path, 'wb') as f:
      for chunk in response.iter_content(1024):
        f.write(chunk)
  except Exception as e:
    print(f"Error downloading {image_url}: {e}")

output_folder = "./Hyundai"
image_name_counter = 595
for img_url in image_urls:
  download_image(img_url, output_folder, image_name_counter, "Hyundai")
  image_name_counter += 1